# 0. Installation

In [6]:
!pip install ipython-sql

# 1. Introduction

In [10]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

# 2. Overview of the Data

In [18]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
(sqlite3.OperationalError) no such table: facts
[SQL: SELECT *
  FROM facts
 LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### Here are the descriptions for some of the columns:

###### name — the name of the country.
###### area — the total land and sea area of the country.
###### population — the country's population.
###### population_growth— the country's population growth as a percentage.
###### birth_rate — the country's birth rate, or the number of births a year per 1,000 people.
###### death_rate — the country's death rate, or the number of death a year per 1,000 people.
###### area— the country's total area (both land and water).
###### area_land — the country's land area in square kilometers.
###### area_water — the country's water area in square kilometers.
#### Let's start by calculating some summary statistics and see what they tell us.

# 3. Summary Statistics

In [19]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

 * sqlite:///factbook.db
(sqlite3.OperationalError) no such table: facts
[SQL: SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### A few things are interesting in the summary statistics on the previous screen:

###### -There's a country with a population of 0.
###### -There's a country with a population of 7256490011 (or more than 7.2 billion people).
#### Let's use subqueries to concentrate on these countries without using the specific values.

# 4. Exploring Outliers

In [20]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
(sqlite3.OperationalError) no such table: facts
[SQL: SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for Antarctica:



In [21]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
(sqlite3.OperationalError) no such table: facts
[SQL: SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

#### Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

# 5. Summary Statistics Revisited


In [22]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
(sqlite3.OperationalError) no such table: facts
[SQL: SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### There's a country whose population closes in on 1.4 billion!

# 6. Exploring Average Population and Area

#### Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

#### We should discard the row for the whole planet.


In [23]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
(sqlite3.OperationalError) no such table: facts
[SQL: SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### We see that the average population is around 32 million and the average area is 555 thousand square kilometers.



# 7. Finding Densely Populated Countries

#### To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have the following:

###### -Above-average values for population.
###### -Below-average values for area.

In [25]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
);

 * sqlite:///factbook.db
(sqlite3.OperationalError) no such table: facts
[SQL: SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### Some of these countries are generally known to be densely populated, so we have confidence in our results!